# Dimensionality

## IKEA 3D

### imports

In [1]:
!rm -rf mola
!git clone https://github.com/dbt-ethz/mola.git

Cloning into 'mola'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1309 (delta 10), reused 9 (delta 2), pack-reused 1281
Receiving objects: 100% (1309/1309), 263.20 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (836/836), done.


In [0]:
from IPython.display import HTML, SVG
import mola.renderBabylonJS as r3D
import random
from mola import grid, graph
import mola.renderP5JS as r2d
import math
from IPython.display import IFrame
import matplotlib.pyplot as plt

### setup

In [0]:
# setup voxel grid and initialise randomly, half full / half empty
nx,ny,nz = 20,10,5
randvalues = [random.randint(0,1) for _ in range(nx*ny*nz)]
randvalues = [v*9+1 for v in randvalues]
my_grid = grid.Grid(nx,ny,nz,values=randvalues)
# create a 3d graph for path calculation
my_graph = graph.Graph.fromGrid3D(nx,ny,nz, mode=1)

In [0]:
def fi(v):
  return v>5
def fo(v):
  return v<5

In [0]:
def compute_path(gr):
  gr.weightFunction = lambda index1,index2: max(my_grid.values[index1],my_grid.values[index2])
  analyser = graph.GraphAnalyser(gr)
  analyser.computeDistancesToNodes([0])
  path = analyser.getShortestPath(nx*ny*nz-1)
  return path

In [0]:
mesh = my_grid.getQuadMesh(fi,fo)

In [7]:
HTML(r3D.displayMesh(mesh))

In [8]:
path = compute_path(my_graph)
print(len(path))

43


In [0]:
def mutate_for_better(old_path):
  ia = random.randint(0,nx*ny*nz-1)
  ib = random.randint(0,nx*ny*nz-1)
  if ia==ib:
    return old_path
  if my_grid.values[ia] == my_grid.values[ib]:
    return old_path
  
  fit0 = len(old_path)
  
  temp = my_grid.values[ia]
  my_grid.values[ia] = my_grid.values[ib]
  my_grid.values[ib] = temp
  
  new_path = compute_path(my_graph)
  fit1 = len(new_path)
  
  out = old_path
  if fit0>fit1:
    #print('was better before')
    temp = my_grid.values[ia]
    my_grid.values[ia] = my_grid.values[ib]
    my_grid.values[ib] = temp
  elif fit0<fit1:
    print('improve! '+str(fit1))
    out = new_path
  else:
    pass
    #print('same')
  return out

In [10]:
for i in range(500):
  #print('i '+str(i))
  path = mutate_for_better(path)

improve! 45
improve! 47
improve! 49
improve! 51
improve! 55


In [11]:
mesh = my_grid.getQuadMesh(fi,fo)
HTML(r3D.displayMesh(mesh))

In [12]:
print('step\tindex\tX Y Z')
for i,p in enumerate(path):
  print(i,'\t',p,'\t',my_grid.getX(p),my_grid.getY(p),my_grid.getZ(p))

step	index	X Y Z
0 	 999 	 19 9 4
1 	 949 	 18 9 4
2 	 944 	 18 8 4
3 	 939 	 18 7 4
4 	 989 	 19 7 4
5 	 988 	 19 7 3
6 	 987 	 19 7 2
7 	 986 	 19 7 1
8 	 985 	 19 7 0
9 	 935 	 18 7 0
10 	 940 	 18 8 0
11 	 890 	 17 8 0
12 	 840 	 16 8 0
13 	 790 	 15 8 0
14 	 740 	 14 8 0
15 	 741 	 14 8 1
16 	 736 	 14 7 1
17 	 737 	 14 7 2
18 	 687 	 13 7 2
19 	 637 	 12 7 2
20 	 587 	 11 7 2
21 	 582 	 11 6 2
22 	 532 	 10 6 2
23 	 527 	 10 5 2
24 	 477 	 9 5 2
25 	 476 	 9 5 1
26 	 426 	 8 5 1
27 	 425 	 8 5 0
28 	 375 	 7 5 0
29 	 325 	 6 5 0
30 	 330 	 6 6 0
31 	 331 	 6 6 1
32 	 336 	 6 7 1
33 	 286 	 5 7 1
34 	 236 	 4 7 1
35 	 186 	 3 7 1
36 	 187 	 3 7 2
37 	 188 	 3 7 3
38 	 183 	 3 6 3
39 	 178 	 3 5 3
40 	 179 	 3 5 4
41 	 129 	 2 5 4
42 	 124 	 2 4 4
43 	 119 	 2 3 4
44 	 69 	 1 3 4
45 	 64 	 1 2 4
46 	 59 	 1 1 4
47 	 54 	 1 0 4
48 	 53 	 1 0 3
49 	 52 	 1 0 2
50 	 2 	 0 0 2
51 	 7 	 0 1 2
52 	 6 	 0 1 1
53 	 5 	 0 1 0
54 	 0 	 0 0 0


![alt text](https://scontent-frx5-1.cdninstagram.com/vp/1a9dd74dcaf0c218d2215b1e1ad3526c/5D306D6F/t51.2885-15/sh0.08/e35/s640x640/54204718_361509934456705_294070234308096797_n.jpg?_nc_ht=scontent-frx5-1.cdninstagram.com)

## Numbers

### Number Game
- 20 \* 10 \* 5 = 1'000 cells
- each either solid or void
- number of possibilities > 2<sup>1'000</sup> (not a number)
- number of possibilities with space half filled:
 - 1000!/(500!\*500!)
- even polyomino with only 3 \* 3 \* 3 = 27 voxels
- 2<sup>27</sup> = 134'217'728 different polyominos

Impossible to exhaustively try all the solutions.

### IKEA
1. start with one random configuration
1. change two pixels and measure again
1. if better, continue from there
1. else switch back

### Search strategies
- Hill climbing: https://en.wikipedia.org/wiki/Hill_climbing
- Simulated annealing: https://en.wikipedia.org/wiki/Simulated_annealing
- more advanced: https://en.wikipedia.org/wiki/Evolutionary_algorithm

## Beijing simplified


![Beijing bird's nest](https://i.pinimg.com/originals/ca/e8/19/cae819357a8dd1c566775f0c90a89296.png)

In [13]:
r2d.beginDraw()
for i in range(100):
  sa = random.random()*math.pi*2
  ea = random.random()*math.pi*2
  r2d.line(280+270*math.cos(sa),240+230*math.sin(sa),
           280+270*math.cos(ea),240+230*math.sin(ea))
HTML(r2d.endDraw())

### Ingredients
- 100 lines
- start angle, end angle

### Fitness
- homogenous distribution
- central void
- fields not too big (cover material)
- fields not too small (mounting)
- beams not too long (bending)

### Encoding
- each solution needs 200 floats (0 ... 2π)
- 200-dimensional vector
- point in 200-dimensional space

### Strategy
1. generate 100 random solutions
- measure fitness
- take best 10
- randomly mutate them to create 100 new
- repeat form #2

![alt text](http://www.kaisersrot.ch/kaisersrot-02/2003_National_Beijing_Olympic_Stadium_%28CN%29_files/serie_all12.png)
http://www.kaisersrot.ch/kaisersrot-02/2003_National_Beijing_Olympic_Stadium_%28CN%29.html

### Endless Forms
- http://endlessforms.com/
- 20 x 20 x 40 = 16'000 dimensional vector

### Fitness Landscape
![alt text](https://upload.wikimedia.org/wikipedia/commons/e/ea/Visualization_of_two_dimensions_of_a_NK_fitness_landscape.png)
https://en.wikipedia.org/wiki/Fitness_landscape